# Building confidence on explainability methods

Local explainability methods are recent, they keep being improved and research is very active in the field. Thus, one needs to be careful when interpreting those methods' outputs. 

Depending on the tasks and data, explainability methods may give different results. The purpose of the metrics presented below is to assess the degree of confidence in these cases. We will answer the following questions: 

- Do different explainability methods give similar explanations on average ? (**Consistency**)

- Is the explanation similar for similar instances ? (**Local stability**)

- Do a few features drive the model ? (**Approximation**)

<b>This short tutorial </b>presents an example of how those 3 metrics could be used on a classification case.

We used Kaggle's [Titanic](https://www.kaggle.com/c/titanic/data) dataset

In [ ]:
import pandas as pd
from category_encoders import OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

In [ ]:
import sys
sys.path.append('/Users/francescomallardmarini/Documents/Data_Analyst_FR/SoGé/Teletravail/MAIF/shapash/')

## Building Supervized Model

Let's start by loading a dataset and building a model that we will try to explain right after.


Load Titanic data

In [ ]:
from shapash.data.data_loader import data_loading
titanic_df, titanic_dict = data_loading('titanic')
del titanic_df['Name']
y_df=titanic_df['Survived'].to_frame()
X_df=titanic_df[titanic_df.columns.difference(['Survived'])]

In [ ]:
titanic_df.head()

Load Titanic data

In [ ]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

Train / Test Split + model fitting

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=7)

# Subsample
Xtrain = Xtrain[:50].reset_index(drop=True)
ytrain = ytrain[:50].reset_index(drop=True)

In [ ]:
clf = ExtraTreesClassifier(n_estimators=200).fit(Xtrain, ytrain)

## Select an explainability method

Now that the model is ready, we need to pick an explainability method. As mentioned earlier, many of them exist, with different theoretical foundation, underlying assumptions, and levels of maturity. Thus, results might differ significantly among methods. Can we trust them?

The **Consistency metric** compares methods between them and evaluates how close the explanations are from each other: if underlying assumptions lead to similar results, we would be more confident in using those methods. If not, careful conideration should be taken in the interpretation of the results

#### First, we need to instantiate and compile the Consistency object

In [ ]:
from explainer.consistency import Consistency

In [ ]:
cns = Consistency()
cns.compile(x=Xtrain, # Dataset for which we need explanations
            model=clf, # Model to explain
            preprocessing=encoder, # Optional
            )

#### We can now display the consistency plot:

In [ ]:
cns.consistency_plot()

For each pair of explainability methods, the metric compares the explanations across the dataset (or a sample if specified) and calculates an average distance between the explainability methods. Two plots are proposed:

* the first graph shows the aforementioned distances between methods on a 2D plan.
    * As we can see here, "shap" and "acv" have very similar explanations, while "lime" provides consistently different explanations.

* the second graph serves as a support for the first one: it gives a better sense of what the distances mean (what is 1.42? is it acceptable or not?). To do so, the metric extracts 5 real comparisons from the dataset (examples are represented by their Id in the dataframe) with distances similar to those in the first plot.
    * As we can see, a distance of .04 gives very similar contributions between features, while 1.44 pretty much describes opposite behaviors.

Depending on the selected methods, examples above show how different the contributions could be. Interpreting them must be done carefully.

Two parameters can be chosen in those graphs:
* _selection:_ a sample of the dataset on which to evaluate the metric expressed as a list of indices (by default take the whole dataset if not too big)

* *max_features*: the number of features displayed in the graph (the most significant ones are selected)

In [ ]:
index = [45, 36, 12, 17, 29]
cns.consistency_plot(selection=index, max_features=5)

_Note:_ Instead of providing a dataset with a model, we can also calculate contributions beforehand and use those ones in the metric using the _contributions_ argument. This allows, for example, to use explainability methods that are not supported in Shapash. Let's for example calculate contributions separately:

In [ ]:
import shap

treeSHAP = shap.TreeExplainer(clf).shap_values(Xtrain[:50], check_additivity=False)[1]
samplingSHAP = shap.SamplingExplainer(clf.predict_proba, shap.kmeans(Xtrain, 10)).shap_values(Xtrain[:50], check_additivity=False)[1]
kernelSHAP = shap.KernelExplainer(clf.predict_proba, shap.kmeans(Xtrain, 10)).shap_values(Xtrain[:50], check_additivity=False)[1]

treeSHAP = pd.DataFrame(treeSHAP, columns=Xtrain[:50].columns)
samplingSHAP = pd.DataFrame(samplingSHAP, columns=Xtrain[:50].columns)
kernelSHAP = pd.DataFrame(kernelSHAP, columns=Xtrain[:50].columns)

In [ ]:
# The format must be a dictionary where keys are methods names and values are pandas DataFrames: be careful to have dataframes with same shape, index and column names
contributions = {"tree\nSHAP": treeSHAP, "sampling\nSHAP": samplingSHAP, "kernel\nSHAP":kernelSHAP}

cns.compile(contributions=contributions)
cns.consistency_plot(selection=index, max_features=5)

## Evaluate the compacity of the explanations

What if the model uses tens or hundreds of features? Is it still going to be easy to understand how it works? Probably not. Indeed, the number of features greatly affects explainability

The **Compacity metric** measures how well each decision can be explained by relying on relatively few features, which may however be different from one instance to another

The idea is the following: _for each instace, we select the features with the highest contributions and we look at how well they approximate the model. Results are then aggregated across the whole dataset (or a sample of it) and displayed_

#### First, we need to instantiate and compile a SmartExplainer

In [ ]:
from explainer.smart_explainer import SmartExplainer

In [ ]:
response_dict = {0: 'Death', 1:' Survival'}

In [ ]:
xpl = SmartExplainer(features_dict=titanic_dict, # Optional parameters
                     label_dict=response_dict) # Optional parameters, dicts specify labels 

In [ ]:
xpl.compile(
    x=Xtrain, # Dataset for which we need explanations
    model=clf, # Model to explain
    preprocessing=encoder, # Optional
)

#### We can now display the consistency plot:

In [ ]:
xpl.plot.compacity_plot()

We want to measure how well the sum of the most important contributions approximates the global decision of the model. The two graphs show the link between the level of approximation, the number of required features to reach it and the proportion of the dataset on which it works.

* In the left graph, for example, we can read that top 4 features reach the default approximation for 93.3% of the instances. Thus, if we needed to provide accurate explanations about the model, a small subset of features will provide a reliable explanation for a vast majority of instances

* In the right graph, for example, we can read that top 5 features reach 99% of the reference model for 28.6% of the instances. Thus, if we needed something extremely precise, we would probably need to consider all features in the explanations

Multiple parameters can be modified, such as:

* *selection*: a sample of the dataset on which to evaluate the metric expressed as a list of indices (by default take the whole dataset if not too big)

* *approx*: how close we want to be the reference model with all features (default 90%) -- Left graph

* *nb_features*: how many features are selected to evaluate the approximation (default 5) -- Right graph

In [ ]:
index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

xpl.plot.compacity_plot(selection=index, approx=.85, nb_features=3)

## Evaluate the stability of the explanations

To increase confidence in the explanation, measuring their stability is important.

We define stability as follows: _if instances are very similar, then one would expect the explanations to be similar as well.
Therefore, locally stable explanations are an important factor that help build trust around a particular explanation._

The similarity between instances is evaluated under two criteria: (1) the instances must be close in the feature space and (2) have similar model outputs.

#### We can now display the stability plot:

In [ ]:
xpl.plot.stability_plot()

The above plot looks at the neighborhood around each provided instance (reminder: neighborhood in terms of features and model output) and shows:

* the average importance of the feature across the dataset based on its contributions (y-axis)

* the average variability of the feature across the instances' neighborhood (x-axis)

Left features are stable in the neighborhood, unlike those on the right. Top features are important, unlike bottom ones

* Here, features like "Sex", "Title" and "Class" in the left-hand side seem to have strong and relatively stable contributions, so one might be more confident in using them for explanations.
* On the other hand, features like "Fare" and "Port of embarcation" are much more unstable, and we might want to be careful before interpreting explanations around those features

Multiple parameters can be modified, such as:

* *selection*: a sample of the dataset on which to evaluate the metric expressed as a list of indices (by default take the whole dataset if not too big)

* *max_features*: the number of features displayed in the graph (the most significant ones are selected)

* *distribution*: changes the type of displayed graph

**Note:** Means are not always representative of the individual behaviors. Thus, we might look at the distribution of variability as well:

In [ ]:
index = [45, 36, 12, 17, 29]

# Using distribution = "boxplot" or "violin" displays distributions of variability instead of means
xpl.plot.stability_plot(selection=index, max_features=5, distribution="boxplot")

Stability can also be evaluated for a single instance. In that case, contributions are directly compared against neighbors and displayed:

In [ ]:
xpl.plot.local_neighbors_plot(index=13)

We can see here how difficult it is to interpret contributions sometimes: the feature _Port of embarkation_ gives completely opposite suggestions in the same neighborhood.